### Red neuronal profunda usando word2vec

In [69]:
import pandas as pd 
import sklearn as sk
import numpy as np

import re
import string
import nltk
from nltk.corpus import stopwords

from time import time
from collections import defaultdict
from tensorflow import keras

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,GRU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

/home/tomas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tomas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tomas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tomas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/tomas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/tomas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/tomas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
english_stopwords=set(stopwords.words('english'))

In [7]:
train_set=pd.read_csv("train.csv",encoding='utf-8')

In [8]:
test_set=pd.read_csv("test.csv",encoding='utf-8')

#####  Procesamiento del texto

In [9]:
def quitar_stopwords(texto):
    texto=' '.join([word for word in texto.split() if word not in english_stopwords])
    return texto

In [10]:
def quitar_menciones(texto):
    texto=' '.join([palabra for palabra in texto.split() if '@' not in palabra])
    return texto

In [11]:
Diccionario_de_lenguaje_de_internet={
"2day": "Today",
"2moro": "Tomorrow",
"2nite": "Tonight",
"4EAE": "For Ever And Ever",
"ABT":"About",
"ADN": "Any Day Now",
"AFAIC": "As Far As I’m Concerned",
"AFAICT": "As Far As I Can Tell",
"AFAIK": "As Far As I Know",
"AFAIR": "As Far As I Remember",
"AKA": "Also Known As",
"AMA": "Ask Me Anything",
"ASAIC": "As Soon As I Can",
"ASAP": "As Soon As Possible",
"ATM": "At The Moment",
"B4": "Before",
"B4N": "Bye For Now",
"Bae": "Babe/Before Anyone Else",
"BBL": "Be Back Later",
"BBT": "Be Back Tomorrow",
"BCNU": "Be Seeing You",
"BD": "Big Deal",
"BF": "Boyfriend",
"BFF": "Best Friends Forever",
"BMT": "Before My Time",
"BOL": "Be On Later",
"BOT": "Back On Topic",
"BRB": "Be Right Back",
"BRO": "Brother",
"BT": "But",
"BTW": "By The Way",
"CFY": "Calling For You",
"CU": "See You",
"CUL": "See You Later",
"Cuz": "Because",
"CYA": "Cover Your Ass",
"DAE": "Does Anyone Else",
"DBA": "Doing Business As",
"DFTBA": "Don’t Forget To Be Awesome",
"DIKU": "Do I Know You",
"DM": "Direct Message",
"DND": "Do Not Disturb",
"DR": "Double Rainbow",
"DWBH": "Don’t Worry, Be Happy",
"ELI5": "Explain Like I’m 5",
"EOM": "End Of Message",
"EOS": "End Of Story",
"F2F": "Face To Face",
"FAQ": "Frequently Asked Question",
"FB": "Facebook",
"FBF": "Flash Back Friday",
"FF": "Follow Friday",
"FIFY": "Fixed It For You",
"FITB": "Fill In The Blank",
"FML": "Fuck My Life",
"FOMO": "Fear Of Missing Out",
"FTFY": "Fixed That For You",
"FTL": "For The Loss",
"FTW": "For The Win",
"FWB": "Friends With Benefits",
"FWIW": "For What It’s Worth",
"FYE": "For Your Entertainment",
"FYEO": "For Your Eyes Only",
"FYI": "For Your Information",
"GA": "Go Ahead",
"GAL": "Get A Life",
"GF": "Girlfriend",
"GM": "Good Morning",
"GN": "Good Night",
"Gr8": "Great",
"GTR": "Getting Ready",
"HAND": "Have A Nice Day",
"HB": "Hurry Back",
"HBD": "Happy Birthday",
"HBU": "How About You",
"HMB": "Hit Be Back",
"HMU": "Hit Me Up",
"HRU": "How Are You",
"HTH": "Hope This Helps",
"IAC": "In Any Case",
"IC": "I See",
"ICYMI": "In Case You Missed It",
"IDC": "I Don’t Care",
"IDK": "I Don’t Know",
"IG": "Instagram",
"IIRC": "If I Remember Correctly",
"IKR": "I Know Right",
"ILY": "I Love You",
"IMHO": "In My Humble Opinion",
"IMMD": "It Made My Day",
"IMY": "I Miss You",
"IRL": "In Real Life",
"IS": "I’m Sorry",
"ISO": "In Search Of",
"IU2U": "It’s Up To You",
"J4F": "Just For Fun",
"JAM": "Just A Minute",
"JFY": "Just For You",
"JIC": "Just In Case",
"JK": "Just Kidding",
"JSYK": "Just So You Know",
"KK": "Okay",
"L8": "Late",
"L8R": "Later",
"LMA": "Leave Me Alone",
"LMAO": "Laughing My Ass Off",
"LMBO": "Laughing My Butt Off",
"LMK": "Let Me Know",
"LOL": "Laugh Out Loud",
"LTNS": "Long Time No See",
"LYLAS": "Love You Like A Sister",
"M/F": "Male or Female",
"M8": "Mate",
"MP": "My pleasure",
"MSM": "Mainstream Media",
"MU": "Miss You",
"MYOB": "Mind Your Own Business",
"NAGI": "Not A Good Idea",
"NBD": "No Big Deal",
"NE1": "Anyone",
"NM": "Not Much",
"NP": "No Problem",
"NSFL": "Not Safe For Life",
"NSFW": "Not Safe For Work",
"NTS": "Note To Self",
"NVM": "Never Mind",
"OC": "Original Content",
"OH": "Overheard",
"OIC": "Oh ! I See",
"OMD": "Oh My Damn",
"OMG": "Oh My Goodness",
"OMW": "On My Way",
"OT": "Off Topic",
"OFC": "Of course",
"PAW": "Parents Are Watching",
"Pls": "Please",
"POTD": "Photo Of The Day",
"POV": "Point Of View",
"PPL": "People",
"PTB": "Please Text Back",
"Q4U": "Question For You",
"QQ": "Crying",
"RBTL": "Read Between The Lines",
"RIP": "Rest In Peace",
"RL": "Real Life",
"ROFL": "Rolling On the Floor Laughing",
"RT": "Retweet",
"RTM": "Read The Manual",
"SIS": "Sister",
"SITD": "Still In The Dark",
"SM": "Social Media",
"SMH": "Shaking My Head",
"SMY": "Somebody",
"SNH": "Sarcasm Noted Here",
"SOL": "Sooner Or Later",
"Some1": "Someone",
"SRSLY": "Seriously",
"STBY": "Sucks To Be You",
"Str8": "Straight",
"SYS": "See You Soon",
"TBA": "To Be Announced",
"TBH": "To Be Honest",
"TBT": "Throwback Thursday",
"TBT": "Truth Be Told",
"TFH": "Thread From Hell",
"TFTI": "Thanks For The Invite",
"TGIF": "Thank God It’s Friday",
"THX": "Thanks",
"TIA": "Thanks in Advance",
"TIL": "Today I Learned",
"TIME": "Tears In My Eyes",
"TL;DR": "Too Long; Didn’t Read",
"TLDR":"Too long didn’t read",
"TL DR":"Too long didn’t read",
"TLC": "Tender Loving Care",
"TMI": "Too Much Information",
"TTYL": "Talk To You Later",
"TTYS": "Talk To You Soon",
"Txt": "Text",
"TYVM": "Thank You Very Much",
"U": "You",
"U4F": "You Forever",
"UR": "Your",
"VBG": "Very Big Grin",
"VSF": "Very Sad Face",
"WB": "Welcome Back",
"WBU": "What About You?",
"WEG": "Wicked Evil Grin",
"WKND": "Weekend",
"WOM": "Word of Mouth",
"WOTD": "Word Of The Day",
"Wru": "Who Are You",
"WTH": "What The Heck?",
"WTPA": "Where The Party At?",
"WU?": "What's Up",
"WU":"What's Up",
"WYCM": "Will You Call Me?",
"WYWH": "Wish You Were Here",
"XOXO": "Hugs and Kisses",
"YGM": "You’ve Got Mail",
"YNK": "You Never Know",
"YOLO": "You Only Live Once",
"YT": "YouTube",
"YW": "You’re Welcome",
}

In [12]:
def reemplazar_lenguaje_internet(texto):
    texto=texto.upper()
    palabras=texto.split()
    palabras_procesadas=[]
    for palabra in palabras:
        traduccion=Diccionario_de_lenguaje_de_internet.get(palabra,'not internet slang')
        if(traduccion!='not internet slang'):
            lista_aux=traduccion.split()
            for x in lista_aux:
                palabras_procesadas.append(x.lower())
        else:
            palabras_procesadas.append(palabra.lower())
    texto=' '.join([word for word in palabras_procesadas])
    return texto

In [13]:
diccionario_contracciones = {  "ain't": "is not",
                               "aint":"is not",
                               "aren't": "are not",
                               "arent":"are not",
                               "can't": "can not",
                               "cant":"can not",
                               "cause": "because",
                               "could've": "could have",
                               "couldve":"could have",
                               "couldn't": "could not",
                               "couldnt":"could not",
                               "didn't": "did not", 
                               "didnt":"did not",
                               "doesn't": "does not",
                               "doesnt":"does not",
                               "don't": "do not",
                               "dont":"do not",
                               "hadn't": "had not",
                               "hadnt":"had not",
                               "hasn't": "has not",
                               "hasnt":"has not",
                               "haven't": "have not",
                               "havent":"have not",
                               "he'd": "he would",
                               "hed":"he would",
                               "he'll": "he will",
                               "he ll":"he will",
                               "he's": "he is",
                               "hes":"he is",
                               "how'd": "how did",
                               "howd":"how did",
                               "how'd'y": "how do you",
                               "howdy":"how do you",
                               "how'll": "how will",
                               "howll":"how will",
                               "how's": "how is",
                               "hows":"how is",
                               "I'd": "I would",
                               "id":"i would",
                               "I'd've": "I would have",
                               "idve":"i would have",
                               "I'll": "I will", 
                               "Ill":"i will", #duda: si algun tweet habla de un enfermo puede traer ruido
                               "I'll've": "I will have",
                               "I'm": "I am", 
                               "im":"i am",
                               "I've": "I have",
                               "ive":"i have",
                               "i'd": "i would",
                               "i'd've": "i would have",
                               "i'll": "i will",
                               "i'll've": "i will have",
                               "i'm": "i am", 
                               "im":"i am",
                               "i've": "i have",
                               "isn't": "is not",
                               "isnt":"is not",
                               "it'd": "it would",
                               "itd":"it would",
                               "it'd've": "it would have",
                               "it'll": "it will",
                               "itll":"it will",
                               "it'll've": "it will have",
                               "it's": "it is",
                               "its":"it is",
                               "let's": "let us",
                               "lets":"let us",
                               "ma'am": "madam",
                               "maam":"madam",
                               "mayn't": "may not",
                               "maynt":"may not",
                               "might've": "might have",
                               "mightve":"might have",
                               "mightn't": "might not",
                               "mightnt":"might not",
                               "mightn't've": "might not have",
                               "must've": "must have",
                               "mustve":"must have",
                               "mustn't": "must not",
                               "mustnt":"must not",
                               "mustn't've": "must not have",
                               "needn't": "need not",
                               "neednt":"need not",
                               "needn't've": "need not have",
                               "o'clock": "of the clock",
                               "oclock":"of the clock",
                               "oughtn't": "ought not",
                               "oughtnt":"ought not",
                               "oughtn't've": "ought not have",
                               "shan't": "shall not",
                               "shant":"shall not",
                               "sha'n't": "shall not",
                               "shan't've": "shall not have",
                               "she'd": "she would",
                               "shed":"she would",
                               "she'd've": "she would have",
                               "she'll": "she will",
                               "shell":"she will",#nuevamente aca tengo ruido por dos palabras igual escritas
                               "she'll've": "she will have", 
                               "she's": "she is",
                               "shes":"she is",
                               "should've": "should have",
                               "shouldve":"should have",
                               "shouldn't": "should not",
                               "shouldnt": "should not",
                               "shouldn't've": "should not have",
                               "so've": "so have",
                               "so's": "so as",
                               "this's": "this is",
                               "that'd": "that would",
                               "that'd've": "that would have",
                               "that's": "that is",
                               "thats":"that is",
                               "there'd": "there would",
                               "thered":"there would",
                               "there'd've": "there would have",
                               "there's": "there is",
                               "theres":"there is",
                               "here's": "here is",
                               "heres":"here is",
                               "they'd": "they would",
                               "theyd":"they would",
                               "they'd've": "they would have",
                               "they'll": "they will",
                               "theyll":"they will",
                               "they'll've": "they will have", 
                               "they're": "they are",
                               "theyre":"they are",
                               "they've": "they have", 
                               "theyve":"they have",
                               "to've": "to have", 
                               "tove":"to have",
                               "wasn't": "was not",
                               "wasnt":"was not",
                               "we'd": "we would",
                               "wed":"we would",#aca nuevamente hay conflicto
                               "we'd've": "we would have",
                               "we'll": "we will",
                               "we'll've": "we will have",
                               "we're": "we are",
                               "we've": "we have",
                               "weren't": "were not",
                               "werent":"were not",
                               "what'll": "what will",
                               "whatll":"what will",
                               "what'll've": "what will have",
                               "what're": "what are", 
                               "whatre":"what are",
                               "what's": "what is",
                               "whats":"what is",
                               "what've": "what have",
                               "whatve":"what have",
                               "when's": "when is",
                               "whens":"when is",
                               "when've": "when have",
                               "whenve":"when have",
                               "where'd": "where did",
                               "whered":"where did",
                               "where's": "where is",
                               "wheres":"where is",
                               "where've": "where have",
                               "whereve":"where have",
                               "who'll": "who will", 
                               "wholl":"who will",
                               "who'll've": "who will have",
                               "who's": "who is",
                               "whos":"who is",
                               "who've": "who have",
                               "whove":"who have",
                               "why's": "why is",
                               "whys":"why is",
                               "why've": "why have",
                               "whyve":"why have",
                               "will've": "will have",
                               "willve":"will have",
                               "won't": "will not",
                               "wont":"will not",
                               "won't've": "will not have",
                               "would've": "would have",
                               "wouldve":"would have",
                               "wouldn't": "would not",
                               "wouldnt":"would not",
                               "wouldn't've": "would not have",
                               "y'all": "you all",
                               "yall":"you all",
                               "y'all'd": "you all would",
                               "yalld":"you all would",
                               "y'all'd've": "you all would have",
                               "y'all're": "you all are",
                               "yallre":"you all are",
                               "y'all've": "you all have",
                               "you'd": "you would",
                               "youd":"you would",
                               "you'd've": "you would have",
                               "you'll": "you will",
                               "youll":"you will",
                               "you'll've": "you will have",
                               "you're": "you are",
                               "youre":"you are",
                               "you've": "you have",
                               "youve":"you have"}


In [14]:
def reemplazar_contracciones(texto):
    texto=texto.lower()
    palabras=texto.split()
    palabras_procesadas=[]
    for palabra in palabras:
        traduccion=diccionario_contracciones.get(palabra,'no es contraccion')
        if(traduccion!='no es contraccion'):
            lista_aux=traduccion.split()
            for x in lista_aux:
                palabras_procesadas.append(x.lower())
        else:
            palabras_procesadas.append(palabra.lower())
    texto=' '.join([word for word in palabras_procesadas])
    return texto

In [15]:
#es necesario importar la libreria re (regular expression) para poder usar esta funcion
#ya esta bajada en el comienzo del notebook pero si lo que quieren es analizar la documentación
#lo pueden revisar en google
#falta borrar los arroba
def limpiar_texto(texto):
    texto = reemplazar_lenguaje_internet(texto)
    texto=reemplazar_contracciones(texto)
    texto=texto.lower()
    texto = re.sub('\[.*?\]', '', texto) # remove text in square brackets
    texto = re.sub('https?://\S+|www\.\S+', '', texto) # remove URLs
    texto = re.sub('<.*?>+', '', texto) # remove html tags
    texto = re.sub('[%s]' % re.escape(string.punctuation), '', texto) # remove punctuation
    texto = re.sub('\n', '', texto) # remove words conatinaing numbers
    texto = re.sub('\w*\d\w*', '', texto)
    texto = re.sub('[‘’“”…]', '', texto)
    texto = quitar_stopwords(texto)
    texto = quitar_menciones(texto)

    return texto

In [16]:
train_set['text']=train_set['text'].apply(limpiar_texto)

In [18]:
from nltk import WordNetLemmatizer

In [19]:
lemmatizer=WordNetLemmatizer()

In [20]:
def lemmatizar_texto(texto):
    lista_palabras=nltk.word_tokenize(texto)
    texto=' '.join([lemmatizer.lemmatize(word) for word in lista_palabras])
    return texto
#nota: la función esta definida aca, pero para ver los resultados que produce hay
#que ir al comienzo del notebook y ejecutar la celda de lemmatizar texto luego
#de haber limpiado el texto original

In [21]:
from nltk.corpus import wordnet

def get_wordnet_pos(palabra):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([palabra])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [22]:
def lemmatizar_con_pos(texto):
    lista_palabras=texto.split()
    texto=' '.join(lemmatizer.lemmatize(palabra,get_wordnet_pos(palabra)) for palabra in lista_palabras)
    return texto

In [23]:
train_set['text']=train_set['text'].apply(lemmatizar_con_pos)

In [25]:
from gensim.models.phrases import Phrases, Phraser

In [26]:
sent=[row.split() for row in train_set['text']]

In [58]:
len(sent)

7

In [28]:
phrases=Phrases(sent,min_count=30,progress_per=10000)

In [32]:
bigram=Phraser(phrases)

In [36]:
sentences=bigram[sent]

In [47]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

14305

In [59]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['im_sorry', 'get', 'fire', 'amp', 'like', 'go', 'new', 'via', 'people', 'one']

In [39]:
import multiprocessing
from gensim.models import Word2Vec

In [41]:
cores = multiprocessing.cpu_count()
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [45]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.01 mins


In [48]:
w2v_model.init_sims(replace=True)

In [56]:
w2v_model.wv.most_similar(positive=["california"])

[('hail', 0.17814017832279205),
 ('class', 0.16760270297527313),
 ('mean', 0.1640603244304657),
 ('learn', 0.1531810164451599),
 ('appear', 0.14799894392490387),
 ('thousand', 0.14281725883483887),
 ('movie', 0.14201197028160095),
 ('death', 0.13591092824935913),
 ('trouble', 0.13466258347034454),
 ('day', 0.1295892894268036)]

In [60]:
w2v_model

In [71]:
w2v_model.wv.get_keras_embedding(train_embeddings=False)

ImportError: Please install Keras to use this function